In [1]:
import os
import sys
import datetime
import requests
import pandas as pd
from requests_html import HTML, HTMLSession, AsyncHTMLSession

import time
from selenium import webdriver
import csv
from tqdm import tqdm
import datetime
from utils import get_comment_post_time

In [2]:
# what is this for?
# BASE_DIR = os.path.dirname(__file__)

## load Projects

In [3]:
driver = webdriver.Chrome("/Users/pi/Downloads/chromedriver")  # Optional argument, if not specified will search path.
for cc in ['uk','es','be-fr','be-nl','cz','fr','it','nl','hu','ro','pt','pl','de']:
    driver.get(f'https://www.trnd.com/{cc}')
    page = HTML(html = driver.page_source)
    articles = page.find('.CampaignOverview', first=True)
    projects_url = articles.attrs['href']
    driver.get(projects_url)
    break

/var/folders/6r/1lcv5hpn0z71k7tc5mq76yn40000gn/T/ipykernel_6797/1957236770.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/pi/Downloads/chromedriver")  # Optional argument, if not specified will search path.


In [5]:
def scrape_projects_page():
    articles_parsed = []

    page = HTML(html = driver.page_source)
    articles = page.find('article')
    for i, article in enumerate(articles):
        link_elem = article.find('.campaign-card__link', first=True)
        category_elem = article.find('.campaign-card__category', first=True)
        title_elem = article.find('h1', first=True)
        description_elem = article.find('.campaign-card__paragraph', first=True)
        dates_elems = article.find('.campaign-card__progress', first=True).find('p')

        articles_parsed.append(
            {
                 'project_id' : i+1,
                 'href' : link_elem.attrs['href'] if link_elem else None,
                 'category' : category_elem.text if category_elem else None,
                 'title' : title_elem.text if title_elem else None,
                 'description' : description_elem.text if description_elem else None,
                 'start_date' : dates_elems[0].text if dates_elems else None,
                 'end_date' : dates_elems[1].text if dates_elems else None
            }
        )

    df = pd.DataFrame(articles_parsed)
    df.to_csv(f'./{cc}_projekte.csv', index=None)  

driver = webdriver.Chrome("/Users/pi/Downloads/chromedriver")  # Optional argument, if not specified will search path.
for cc in ['es','cz','be-fr','be-nl','uk','fr','it','nl','hu','ro','pt','pl','de']:
    driver.get(f'https://www.trnd.com/{cc}')
    page = HTML(html = driver.page_source)
    articles = page.find('.CampaignOverview', first=True)
    projects_url = articles.attrs['href']
    driver.get(projects_url)

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)");
        time.sleep(1)

        element = driver.find_element_by_id('load-more')
        if not element.is_displayed():
            break
        else:
            element.click()

        time.sleep(1)

    scrape_projects_page()
    break

/var/folders/6r/1lcv5hpn0z71k7tc5mq76yn40000gn/T/ipykernel_6797/1917391326.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/pi/Downloads/chromedriver")  # Optional argument, if not specified will search path.
/var/folders/6r/1lcv5hpn0z71k7tc5mq76yn40000gn/T/ipykernel_6797/1917391326.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id('load-more')


## load blog_links

In [11]:
df = pd.read_csv(f'./de_projekte.csv')
session = HTMLSession()

csv_blog_line = []
blog_id = 1
for i, row in df.iterrows():
    blog_links = []
    
    project_base_url = row['href']
    print(f'\n#{i+1}  {project_base_url}')
    homepage = session.get(project_base_url)
    
    nav_bar = homepage.html.find('.project-nav',first=True)
    phase_wrapper = homepage.html.find('.phase-wrap',first=True)
    
#     has nav-bar?
    if nav_bar:
        if any([x.text == 'Blog' for x in nav_bar.find('a')]):
            blog_links.extend(get_blog_links_design1(f'{project_base_url}/blog'))
            print(str(blog_links) + "   Blog")
        elif any([x.text == 'Projektblog' for x in nav_bar.find('a')]):
            project_blog_link = [x for x in nav_bar.find('a') if x.text == 'Projektblog'][0].attrs['href']
            blog_links.extend(get_blog_links_design1(project_blog_link))
            print(str(blog_links) + "   Projektblog")
    
    if phase_wrapper:
        blog_links.extend(get_blog_links_design2(phase_wrapper))
        print(str(blog_links) + "   Phases")

    for link in blog_links:
        csv_blog_line.append({
            'project_id' : row['project_id'],
            'blog_id' : blog_id,
            'blog_href' : link

        })
        blog_id+=1
    break
df = pd.DataFrame(csv_blog_line)
df.to_csv('./de_blog_links.csv', index=None)  


#1  https://www.trnd.com/de/projekte/hipp-babysanft-lotion


In [14]:
csv_blog_line

[]

In [ ]:
# PARSE ALL BLOG-LINKS FROM PAGE
def get_blog_links_design1(blog_url):
    blog_page = session.get(blog_url)

#     scrapes first page
    blog_links = []
    blog_links.extend([blog.find('a',first=True).attrs['href'] for blog in blog_page.html.find('article')])
        
#     if additional pages exist, scrape them also
    has_next = blog_page.html.find('.next',first=True)
    while has_next:
        blog_page = session.get(has_next.attrs['href'])
        blog_links.extend([blog.find('a',first=True).attrs['href'] for blog in blog_page.html.find('article')])

        has_next = blog_page.html.find('.next',first=True)

    return blog_links    

def get_blog_links_design2(phase_wrapper):
    blog_links = []
    for phase_link in [phase.attrs['href'] for phase in phase_wrapper.find('a')]:
        blog_page = session.get(phase_link)
        blog_links.extend([blog.find('a',first=True).attrs['href'] for blog in blog_page.html.find('article')])
        
    #     if additional pages exist, scrape them also
        has_next = blog_page.html.find('.next',first=True)
        while has_next:
            blog_page = session.get(has_next.attrs['href'])
            blog_links.extend([blog.find('a',first=True).attrs['href'] for blog in blog_page.html.find('article')])
            
            has_next = blog_page.html.find('.next',first=True)

    return blog_links

In [ ]:
df = pd.DataFrame(csv_comment_lines)
df = df.reindex(['project_id','blog_id','comment_id','comment_author','comment_time','comment_text'],axis=1)
df = df.sort_values(by=['blog_id','comment_id'],ascending=[True,True])
df.to_csv('./de_comments.csv', index=None)  

In [ ]:
df

In [ ]:
df = pd.read_csv(f'./de_blog_links.csv')
session = HTMLSession()

csv_comment_lines = []
# comment_id = 675675
comment_id = 1
for i, row in df.iterrows():
#     if row['blog_id'] <= 2267:
#         continue
    comments_parsed = parse_blog(row['blog_href'])
    print(f'#{len(comments_parsed)}  --  {row["blog_href"]}')
    
    for comment_parsed in comments_parsed:
        comment_parsed['project_id'] = row['project_id']
        comment_parsed['blog_id'] = row['blog_id']
        comment_parsed['comment_id'] = comment_id
        
        csv_comment_lines.append(comment_parsed)
        comment_id+=1

df = pd.DataFrame(csv_comment_lines)
df = df.reindex(['project_id','blog_id','comment_id','comment_author','comment_time','comment_text','reply_to','is_moderator'],axis=1)
df = df.sort_values(by=['blog_id','comment_id'],ascending=[True,True])
# df.to_csv('./de_comments.csv', index=None, mode='a', header=False)  
df.to_csv('./de_comments.csv', index=None)  

In [ ]:
df = pd.DataFrame(csv_comment_lines)
df = df.reindex(['project_id','blog_id','comment_id','comment_author','comment_time','comment_text','reply_to','is_moderator'],axis=1)
df = df.sort_values(by=['blog_id','comment_id'],ascending=[True,True])
# df.to_csv('./de_comments.csv', index=None)  
df

In [ ]:
## debugging
session = HTMLSession()
# blog_url = 'https://www.trnd.com/de/projekte/welde-naturstoff-kellerbier/aktionsphase/vollmundiger-biergenuss-in-der-geschwungenen-flasche'
blog_url = 'https://www.trnd.com/de/projekte/cetaphil-sun-kids/blog/cetaphil-lisa-stellt-sich-vor'
blog_url = 'https://www.trnd.com/de/projekte/tiptoi-2021/blog/gewinnspiel-unsere-reise-um-die-welt'
blog_url_trouble = 'https://www.trnd.com/de/projekte/odolmed3-senses/blog/die-produktreihe'
blog_url_trouble = 'https://www.trnd.com/de/projekte/braun-multiquick7-mq745-aperitif/aktionsphase/unsere-aufgaben'

df = pd.DataFrame(parse_blog(blog_url_trouble))
# df.to_csv('./test.csv', index=None)  
# get_comment_post_time('9 Monate')

In [ ]:
df

In [ ]:
# PARSE ALL COMMENTS FROM BLOG
def parse_blog(blog_url):    
    blog_page = session.get(blog_url)
    
    n_comments = blog_page.html.find('.comment-headline',first=True)
    n_comments = n_comments.text if n_comments != None else '0 Comments'
    print(f"#{n_comments}")
    
    

#         COMMENTS
    comments_parsed = []
    comments_div = blog_page.html.find("#comments",first=True)
    if comments_div == None:
        return []
    else:
        comments_parsed.extend(parse_comments(comments_div))

        has_prev = blog_page.html.find('.prev',first=True)
        while has_prev:
#             print(has_prev.attrs['href'])
            blog_page = session.get(has_prev.attrs['href'])
            comments_div = blog_page.html.find("#comments",first=True)
            comments_parsed.extend(parse_comments(comments_div))
            has_prev = blog_page.html.find('.prev',first=True)

        return comments_parsed

        #     break
#         df = pd.DataFrame(comments_parsed)
#         df.insert(0,'comment_id',range(1,len(comments_parsed)+1))
#         return df

def parse_comments(comments_div):
    comments_parsed = []
    scrape_time = datetime.datetime.now()
    for comment_div in comments_div.xpath('/html/div/div/child::div'):
        
        comments_raw = []
        is_reply = False
        if 'replies' in comment_div.attrs['class']:
            comments_raw = comment_div.find('.comment')
            is_reply = True
        else:
            comments_raw = [comment_div]
        
        for comment in comments_raw:
            comment_author = comment.find('.comment-author',first=True).text
            comment_time = comment.find('.comment-time',first=True).text[6:]
            comment_text = comment.find('p')[1].text if len(comment.find('p')) == 2 else comment.find('dl',first=True).text.strip()
#             comment_text = comment.find('p')[1].text
            is_moderator = len(comment.find('.moderator')) > 0
            reply_to = comment_text.split(': ')[0] if is_reply else None

            comments_parsed.append(
                {
                     'comment_author' : comment_author,
                     'is_moderator' : is_moderator,
                     'comment_time' : str(datetime.datetime.now() - datetime.timedelta(seconds=get_comment_post_time(comment_time))).split('.')[0],
                     'comment_text' : comment_text,
                     'reply_to' : reply_to
                }
            )
    return comments_parsed

## debuggins
# no comments https://www.trnd.com/de/projekte/welde-naturstoff-kellerbier/aktionsphase/vollmundiger-biergenuss-in-der-geschwungenen-flasche
# phases with https://www.trnd.com/de/projekte/lechuza-pflanzgefaesse/aktionsphase/unsere-abschliesende-meinung-zum-lechuza-classico


In [ ]:
#      BLOG-META
#         blogs_parsed = []
#         parse_blog()
#         blog_page_element.html.find('meta')
#         blogs_parsed.append([])